Строим запрос к нашему сервису

In [1]:
import requests
from urllib import request, parse
import json 

In [2]:
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import f1_score


In [3]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [4]:
a = X_test.loc[7]

In [5]:
# формируем запрос
def send_json(x):

    #myurl = 'http://948a-35-230-58-89.ngrok.io/' + '/predict'
    
    myurl = 'http://127.0.0.1:5000/' + '/predict'
    headers = {'content-type': 'application/json; charset=utf-8'}
    response = requests.post(myurl, json=dict(x), headers=headers)
    return response.json()['predictions']

In [6]:
# обращение к серверу с запросом из одного набора (его построили руками выше - data)
response = send_json(a)
print('предсказание', response)

предсказание 0.15281280292096408


In [7]:
X_test.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active
0,61393,19812,2,175,84.0,110,70,1,2,0,0,1
1,56568,18218,1,158,60.0,130,80,3,3,0,0,1
2,93405,21352,1,165,62.0,120,80,1,1,0,0,1
3,77853,21640,1,154,88.0,130,80,3,1,0,0,1
4,82603,22035,1,159,79.0,130,80,1,1,0,0,1


2. Передаем запрос одиночный из наших данных и посмотрим на ответ

In [8]:
response = send_json(X_test.iloc[0,:])

In [9]:
response

0.3776122854473996

3. Сделаем N запросов и оценим время

In [10]:
N = 500

In [11]:
%%time
predictions = X_test.iloc[:N].apply(lambda x: send_json(x), axis=1)

Wall time: 5.79 s


In [12]:
predictions.values[:5]

array([0.37761229, 0.69856775, 0.38277795, 0.82806074, 0.5847539 ])

4. Посчитаем метрику

In [13]:
import numpy as np

precision, recall, thresholds = precision_recall_curve(y_test[:N], predictions)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.3947059520199761, F-Score=0.699, Precision=0.633, Recall=0.781


In [14]:
roc_auc_score(y_score=predictions.values, y_true=y_test.values[:N])

0.763891576367389